# Dollar Cost Averaging (DCA) with cryptocurrencies
A notebook to explore the effectiveness of DCA with different popular cryptocurrencies

In [30]:
import os
import numpy as np 
import pandas as pd 
import datetime as dt
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.metrics import MeanSquaredError

In [31]:
# read data from CSVs, starting with my three favourites
BTC_df = pd.read_csv(".\data\crypto_prices\coin_Bitcoin.csv", parse_dates=['Date'])
ETH_df = pd.read_csv(".\data\crypto_prices\coin_Ethereum.csv", parse_dates=['Date'])
ADA_df = pd.read_csv(".\data\crypto_prices\coin_Cardano.csv", parse_dates=['Date'])

merged_df = BTC_df.append(ETH_df)
merged_df = merged_df.append(ADA_df)
merged_df.shape

(6525, 10)

## BTC DCA Case Study

Starting with a case study example, let's say I put in $100 into bitcoin every week in 2019:
1. how much would that leave me with at the start of 2020? 
2. how would it compare to if I'd just invested it all at once?

In [47]:
df = BTC_df.set_index(['Date'])

def get_btc_price(date):
    # get the bitcoin close price that day
    try:
        price = df.loc[date.strftime("%Y-%m-%d"):date.strftime("%Y-%m-%d")]['Close'].iloc[0]
    except:
        print(date)
    return price

def buy_btc(dollar_amt, date):
    # get the bitcoin close price that day
    price = get_btc_price(date)
    
    # get the bitcoin amount purchased given the dollar amt
    btc_amt = dollar_amt/price

    return btc_amt

Question 1: Gains when 2020 comes around

In [62]:
YEAR = 2021
start = "01/01/{}".format(YEAR)
curr_date = dt.datetime.strptime(start, "%m/%d/%Y")
total_btc = 0.0
money_per_week = 100
weeks = 0

# iterate through each week of the year 
while(curr_date.year == YEAR and curr_date < df.index.max()):
    total_btc += buy_btc(money_per_week, curr_date)
    # print('now own: {} BTC'.format(total_btc))
    curr_date = curr_date + dt.timedelta(days=7)
    weeks += 1

# quick math
curr_date = df.index.max() if curr_date > df.index.max() else  curr_date
curr_btc_price = get_btc_price(curr_date) * total_btc
money_spent = money_per_week*weeks
profit = curr_btc_price - money_spent
increase = profit/money_spent * 100

print('You gained {:.2f} BTC and spent ${} over the course of {} weeks'.format(total_btc, money_spent, weeks))
print('That amount of BTC is worth ${:.2f} on {}'.format(curr_btc_price, curr_date.strftime("%Y/%m/%d")))
print('You earned {:.2f} in profit for an increase of {:.2f}%'.format(profit, increase))

You gained 0.06 BTC and spent $2700 over the course of 27 weeks
That amount of BTC is worth $2148.00 on 2021/07/06
You earned -552.00 in profit for an increase of -20.44%


Question 2: What if we just invested it all at once?

In [64]:
start = "01/01/{}".format(YEAR)
start_date = dt.datetime.strptime(start, "%m/%d/%Y")
total_btc = buy_btc(money_spent, start_date)

# quick math
curr_btc_price = get_btc_price(curr_date) * total_btc
profit = curr_btc_price - money_spent
increase = profit/money_spent * 100

print('You gained {:.2f} BTC and spent ${} at the start of the year'.format(total_btc, money_spent))
print('That amount of BTC is worth ${:.2f} on {}'.format(curr_btc_price, new_date.strftime("%Y-%m-%d")))
print('You earned {:.2f} in profit for an increase of {:.2f}%'.format(profit, increase))

You gained 0.09 BTC and spent $2700 at the start of the year
That amount of BTC is worth $3146.82 on 2021-07-06
You earned 446.82 in profit for an increase of 16.55%
